# IMEC2201 Herramientas Computacionales 
## Semana 3: Interpolación y Curva de Bomba
### Clase 4: Ajuste de Datos e Incertidumbre

Universidad de los Andes — Abril 18, 2022.

---

## TABLA DE CONTENIDO

### Sección 1: Interpolación [→](#section1)
- 1.1. Cargar Librerías
- 1.2. Mínimos Cuadrados Ordinarios (OLS)
    - 1.2.1. Gráfica MBHP vs Masa
    - 1.2.2. Ajuste Lineal
    - 1.2.3. Coeficiente de Correlación
    - 1.2.4. Caso Práctico
- 1.3. Histogramas

### Sección 2: Incertidumbre [→](#section2)
- 2.1. Error Aleatorio
- 2.2. Barras de Error
- 2.3. Librería Measurements.jl
    - 2.3.1. Barras de Error
    - 2.3.2. Propagación de Error
___

**¡Importante!** Note que la carpeta **week3** contiene dos archivos: `Project.toml` y `Manifest.toml`. Estos configuran el <font color="#FF5733">*ambiente de trabajo*</font> y facilitan el manejo de <font color="#FF5733">librerías</font>.

Para inicializar el ambiente desde Julia REPL, dentro de la carpeta **week2**, ejecute lo siguiente:

```shell
$ (v1.7) pkg > activate .
$ (week2) pkg> instantiate
```

**Nota:** Para acceder al Pkg REPL, escriba el corchete derecho `]` en el Julia REPL. Note que el REPL cambie de color de verde (Julia REPL) a azul (Pkg REPL). Una vez ejecutado el código `activate .`, el prefijo entre paréntesis indica el ambiente activo: cambia de `v1.7` a `week2`.

O, desde Jupyter Notebook, ejecute:

```julia
$ using Pkg
$ Pkg.activate()
$ Pkg.instantiate()
```

**Nota:** La activación del ambiente <font color="#FF5733">*precompila*</font> las librerías por lo que puede tardar un momento su ejecución completa.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Puede obtener más información en la documentación oficial de la librería [`Pkg.jl`](https://pkgdocs.julialang.org/v1/environments/) y en el documento dando clic [aquí](https://towardsdatascience.com/how-to-setup-project-environments-in-julia-ec8ae73afe9c).
</div>

In [ ]:
using Pkg
Pkg.activate(pwd())
Pkg.instantiate()

In [ ]:
Pkg.status()

<a id="section1"></a>
# Sección 1: Interpolación

La interpolación es el ejercicio de determinar una función $ f(x) $ que pase por un conjunto de puntos.

Típicamente, en los laboratorios de los cursos de Ingeniería Mecánica, se dispone de un número de datos obtenidos por muestreo a partir de un experimento y se pretende construir una función que los ajuste para así obtener nuevos puntos y precisar/expander el análisis.

**Nota:** La interpolación es diferente a la *regresión* dado que la última busca determinar una función que **describa** el modelo físico particular; es decir, una tendencia.

En general, vamos a utilizar la librería `Interpolations.jl` para explorar diferentes tipos de interpolación.

## 1.1. Cargar Librerías

In [ ]:
using Plots
using Interpolations

# 1.2. Interpolación Lineal

En este escenario, tenemos un par de puntos $(x_k,\:y_k)$ y se quiere conocer el valor de $y$ para un valor cualquiera de $x$ en el intervalo $[x_0, x_1, ..., x_n]$.

El ejercicio consiste en determinar la recta que pasa por dos puntos continuos, es decir, en el intervalo $[x_k, \:x_{k+1}]$.

El modelo matemático para un par de puntos $(x_a,\:y_a)$ y $(x_b,\:y_b)$ es:

$$
y = y_a + (x - x_a) \cdot \frac{y_b - y_a}{x_b - x_a}
$$

Con la librería `Interpolations.jl` utilizamos la función `LinearInterpolation`. La sintaxis de esta función consiste en:

1. Definir el interpolador mediante `LinearInterpolation(x,y)`.
2. Ejecutar la función determinada por el interpolador en el conjunto de puntos independiente mediante.

In [ ]:
# Límite inferior y superior del intervalo
a = 0.0
b = 2π
ndatos = 7

# Definición del intervalo
x = collect(range(a, stop=b, length=ndatos))

# Función que originalmente desconocemos
y = sin.(x)

# Interpolación lineal
itp_lineal = LinearInterpolation(x, y)

# Ejecución del interpolador
f_lineal(x) = itp_lineal(x)

# Gráficas
plot(x, y, seriestype=:scatter, m=:circle, ms=5, mc=:dodgerblue, label="Datos Conocidos", legend=true)
plot!(x, f_lineal, seriestype=:line, ls=:dash, w=2, m=:none, ms=5, label="Interpolación Lineal", legend=true)

En este caso, si quisiera conocer el valor en un punto dado, debo utilizar la misma función, indicando el valor de interés, es decir: `LinearInterpolation(x)`.

In [ ]:
x

In [ ]:
x_interes = 2.5
y_interes = itp_lineal(x_interes)

println("Para x = $(x_interes), el valor de y es $(round(y_interes, digits=2)).")

plot!([x_interes], [y_interes], seriestype=:scatter, m=:circle, ms=5, label="Dato Interés", legend=true)

Note lo que sucede si se aumenta el número de puntos del conjunto de datos conocidos.

In [ ]:
# Intervalo para X_interes
function x_i(a, b, ndatos)
    Xs = range(a, stop=b, length=ndatos)
    xs = []
    
    for i in Xs
        push!(xs, i)
    end
    
    return xs
end

# Intervalo para Y_interes
function y_i(funct, a, b, ndatos)
    Xs = range(a, stop=b, length=ndatos)
    ys = []
    
    for i in Xs
        y = funct(i)
        push!(ys, y)
    end
    
    return ys
end

function funct(x)
    return sin.(x)
end

In [ ]:
ndatos = 7

# Definición del intervalo
x = collect(range(a, stop=b, length=ndatos))

# Función que originalmente desconocemos
y = sin.(x)

# Gráfica teórica
plot(x, y, seriestype=:scatter, m=:circle, ms=5, mc=:dodgerblue, label="Datos Conocidos", legend=true)

# Interés
ndat = 15
x_int = x_i(a, b, ndat)
y_int = y_i(funct, a, b, ndat)

# Interpolación lineal
itp_lineal = LinearInterpolation(x_int, y_int)

# Ejecución del interpolador
f_lineal(x) = itp_lineal(x)

plot(x, y, seriestype=:scatter, m=:circle, ms=5, mc=:dodgerblue, label="Datos Conocidos", legend=true)
plot!(x, f_lineal, seriestype=:line, ls=:dash, w=1, m=:none, ms=5, label="Int. Lin. Pocos Puntos", legend=true)
plot!(x_int, f_lineal, seriestype=:line, ls=:solid, w=1, m=:rect, ms=3, label="Int. Lin. Muchos Puntos", legend=true)

Ahora, si en lugar de partir de una función utilizamos una nube de puntos (como típicamente es en un laboratorio), ¿cómo lo haríamos y qué obtendríamos?

In [ ]:
xx = [0.97, 1.12, 2.92, 3.00, 3.33, 3.97, 6.10, 8.39, 8.56, 9.44]
yy = [2.58, 0.43, 0.06, 5.74, 7.44, 8.07, 6.37, 2.51, 1.44, 0.52]

# Interpolación lineal
itp_lineal = LinearInterpolation(xx, yy)

# Ejecución del interpolador
f_lineal(x) = itp_lineal(x)

# Gráficas
plot(xx, yy, seriestype=:scatter, m=:circle, ms=5, mc=:dodgerblue, label="Datos Conocidos", legend=true)
plot!(xx, f_lineal, seriestype=:line, ls=:dash, w=2, m=:none, ms=5, label="Interpolación Lineal", legend=true)

# 1.3. Interpolación con Splines

Un *spline* es una curva suave definida en partes mediante diferentes polinomios.

En interpolación, los *splines* dan lugar a buenos resultados para formas complicadas requiriendo solamente el uso de polinomios de bajo grado, evitando así las oscilaciones indeseables que se dan al interpolar mediante polinomios de grado elevado.

Con la librería `Interpolations.jl` utilizamos la función `BSplines`. Como mencionamos que los *splines* se definen a partir de polinomios, es necesario establecer el grado de la función (equivalente al grado del polinomio). Con la librería `Interpolations.jl` podemos definir los siguientes grados a los *splines*: `Constant`, `Linear`, `Quadratic`, y `Cubic`, correspondiendo a grados 0, 1, 2 y 3, respectivamente.

In [ ]:
# Límite inferior y superior del intervalo
a = 0.0
b = 2π
ndatos = 7

# Definición del intervalo
x = collect(range(a, stop=b, length=ndatos))

# Función que originalmente desconocemos
y = sin.(x)

# Interpolación lineal
itp_cubic = CubicSplineInterpolation(range(a, stop=b, length=ndatos), y)

# Ejecución del interpolador
f_cubic(x) = itp_cubic(x)

# Gráficas
plot(x, y, seriestype=:scatter, m=:circle, ms=5, mc=:dodgerblue, label="Datos Conocidos", legend=true)
plot!(range(a, stop=b, length=15), f_cubic, seriestype=:line, ls=:dash, w=2, m=:none, ms=5, label="Interpolación Lineal", legend=true)

In [ ]:
x_interes = 2.5
y_interes = itp_cubic(x_interes)

println("Para x = $(x_interes), el valor de y es $(round(y_interes, digits=2)).")

plot!([x_interes], [y_interes], seriestype=:scatter, m=:circle, ms=5, label="Dato Interés", legend=true)

## 1.4. Splines 1D 

In [ ]:
using Dierckx

# Límite inferior y superior del intervalo
a = 0.0
b = 2π
ndatos = 7

# Definición del intervalo
x = collect(range(a, stop=b, length=ndatos))

# Función que originalmente desconocemos
y = sin.(x)

# Spline 1D
spl = Spline1D(x, y)

spl(2.5)

## 1.5. ¿Matemáticamente?

In [ ]:
xlst = collect(range(0, stop=2π, length=7));
flst = sin.(xlst);

# receives number of points n (odd) to interpolate # and the original data via x and f
function interp1(n, x, f)
    xnew = collect(range(x[1], stop=x[end], length=n)); # creates new axes
    fnew = zeros(n); # and initialize new data as zero
    
    i = 1; # label new sites
    for j=1:(length(x)-1) # runs over old sites
        xa = x[j]; # known points
        xb = x[j+1];
        
        fa = f[j]; # known data
        fb = f[j+1];
        
        # matrix form to find the coefficients
        M = [xa 1.0; xb 1.0];
        C1, C0 = inv(M)*[fa; fb];
        # calculate the new data within every two points interval
        while i <= n && xnew[i] <= xb
            fnew[i] = C1*xnew[i] + C0;
            i += 1;
        end
    end
    return xnew, fnew; # return interpolated data
end

# calls function to interpolate
xnew, fnew = interp1(50, xlst, flst);

# plot old data, new data, and exact function
plot(xlst, flst, seriestype=:scatter, m=:circle, ms=4, label="Original points")
plot!(xnew, fnew, seriestype=:line, ls=:dot, m=:rect, ms=2.5, label="Interpolated points")
plot!(xnew, sin.(xnew), seriestype=:line, m=:none, ms=4, label="sin(x)")